**1. Data Exploration and Preprocessing**

In [6]:
import pandas as pd
data= pd.read_csv('blogs.csv')
data

,Data,Labels
0,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism
1,Newsgroups: alt.atheism\nPath: cantaloupe.srv....,alt.atheism
2,Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...,alt.atheism
3,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism
4,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:53...,alt.atheism
...,...,...
1995,Xref: cantaloupe.srv.cs.cmu.edu talk.abortion:...,talk.religion.misc
1996,Xref: cantaloupe.srv.cs.cmu.edu talk.religion....,talk.religion.misc
1997,Xref: cantaloupe.srv.cs.cmu.edu talk.origins:4...,talk.religion.misc
1998,Xref: cantaloupe.srv.cs.cmu.edu talk.religion....,talk.religion.misc


In [7]:
data = data.iloc[:,1:]

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Labels  2000 non-null   object
dtypes: object(1)
memory usage: 15.8+ KB


In [10]:
len(data['Labels'].unique())

20

In [12]:
data.duplicated().sum()

1980

In [13]:
data = data.drop_duplicates()

In [14]:
import nltk
#Downloading necessary files

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import pandas as pd

#Load the data
data = pd.read_csv('blogs.csv')

#printing columns to verify the correct columns names
print("Columns in the dataset:", data.columns)

#Define the text preprocessing function
def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text) #Tokenize the text
    stop_words = set(stopwords.words('english')) #Load stopwords
    filtered_tokens = [word for word in tokens if word not in stop_words] #Remove stopwords
    return ' '.join(filtered_tokens)

# Replace 'Data' with the correct column name after inspecting column names
text_column = 'Data'  # Update with the correct column name if different

if text_column in data.columns:
    data['cleaned_text'] = data[text_column].apply(preprocess_text)
    print(data[['cleaned_text']].head())  # Display the first few rows of the cleaned text
else:
    print(f"Column '{text_column}' not found in the DataFrame. Please check and update the column name.")

Columns in the dataset: Index(['Data', 'Labels'], dtype='object')
                                        cleaned_text
0  path cantaloupesrvcscmuedumagnesiumclubcccmued...
1  newsgroups altatheism path cantaloupesrvcscmue...
2  path cantaloupesrvcscmuedudasnewsharvardedunoc...
3  path cantaloupesrvcscmuedumagnesiumclubcccmued...
4  xref cantaloupesrvcscmuedu altatheism53485 tal...


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer()
x= tfidf_vectorizer.fit_transform(data['cleaned_text'])

# The labels for classification
y = data['Labels']

In [20]:
x

<2000x56432 sparse matrix of type '<class 'numpy.float64'>'
	with 268023 stored elements in Compressed Sparse Row format>

In [21]:
y

0              alt.atheism
1              alt.atheism
2              alt.atheism
3              alt.atheism
4              alt.atheism
               ...        
1995    talk.religion.misc
1996    talk.religion.misc
1997    talk.religion.misc
1998    talk.religion.misc
1999    talk.religion.misc
Name: Labels, Length: 2000, dtype: object

**2. Naive Bayes Model for Text Classification**

In [22]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [23]:
from sklearn.naive_bayes import MultinomialNB

# Initialize the classifier
nb_classifier = MultinomialNB()

# Train the model
nb_classifier.fit(x_train, y_train)



MultinomialNB()

In [24]:
# Make predictions
y_pred = nb_classifier.predict(x_test)
y_pred

array(['comp.sys.mac.hardware', 'sci.crypt', 'sci.crypt', 'rec.autos',
       'talk.politics.mideast', 'sci.crypt', 'soc.religion.christian',
       'soc.religion.christian', 'talk.politics.misc', 'alt.atheism',
       'sci.electronics', 'soc.religion.christian',
       'comp.os.ms-windows.misc', 'comp.graphics', 'rec.autos',
       'talk.politics.misc', 'comp.sys.mac.hardware',
       'talk.religion.misc', 'soc.religion.christian',
       'comp.sys.ibm.pc.hardware', 'alt.atheism', 'sci.electronics',
       'rec.autos', 'soc.religion.christian', 'rec.sport.hockey',
       'sci.med', 'comp.sys.mac.hardware', 'sci.crypt', 'sci.crypt',
       'talk.politics.guns', 'sci.electronics', 'sci.crypt',
       'soc.religion.christian', 'talk.politics.mideast',
       'rec.sport.hockey', 'comp.sys.mac.hardware', 'rec.sport.hockey',
       'comp.graphics', 'comp.sys.ibm.pc.hardware', 'rec.sport.baseball',
       'alt.atheism', 'sci.med', 'sci.electronics', 'alt.atheism',
       'talk.politics.guns'

**3. Sentiment Analysis**

In [25]:
# Install TextBlob
!pip install textblob

# Download TextBlob corpora (for sentiment analysis functionality)
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('brown')

from textblob import TextBlob

# Function to get sentiment
def get_sentiment(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative'

# Apply sentiment analysis
data['sentiment'] = data['cleaned_text'].apply(get_sentiment)

# Display sentiment distribution
print(data['sentiment'].value_counts())


Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 624.3/624.3 kB 2.3 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 7.4 MB/s eta 0:00:00


[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\brown.zip.


positive    1451
negative     546
neutral        3
Name: sentiment, dtype: int64


In [26]:
data

,Data,Labels,cleaned_text,sentiment
0,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism,path cantaloupesrvcscmuedumagnesiumclubcccmued...,positive
1,Newsgroups: alt.atheism\nPath: cantaloupe.srv....,alt.atheism,newsgroups altatheism path cantaloupesrvcscmue...,negative
2,Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...,alt.atheism,path cantaloupesrvcscmuedudasnewsharvardedunoc...,positive
3,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism,path cantaloupesrvcscmuedumagnesiumclubcccmued...,positive
4,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:53...,alt.atheism,xref cantaloupesrvcscmuedu altatheism53485 tal...,positive
...,...,...,...,...
1995,Xref: cantaloupe.srv.cs.cmu.edu talk.abortion:...,talk.religion.misc,xref cantaloupesrvcscmuedu talkabortion120945 ...,positive
1996,Xref: cantaloupe.srv.cs.cmu.edu talk.religion....,talk.religion.misc,xref cantaloupesrvcscmuedu talkreligionmisc837...,positive
1997,Xref: cantaloupe.srv.cs.cmu.edu talk.origins:4...,talk.religion.misc,xref cantaloupesrvcscmuedu talkorigins41030 ta...,positive
1998,Xref: cantaloupe.srv.cs.cmu.edu talk.religion....,talk.religion.misc,xref cantaloupesrvcscmuedu talkreligionmisc836...,positive


**evalution of matrix**

In [27]:
# Calculate evaluation metrics
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

                          precision    recall  f1-score   support

             alt.atheism       0.85      0.61      0.71        28
           comp.graphics       0.83      0.83      0.83        18
 comp.os.ms-windows.misc       0.96      0.92      0.94        24
comp.sys.ibm.pc.hardware       0.53      0.89      0.67        18
   comp.sys.mac.hardware       0.72      0.68      0.70        19
          comp.windows.x       1.00      0.82      0.90        22
            misc.forsale       0.72      0.81      0.76        16
               rec.autos       1.00      1.00      1.00        21
         rec.motorcycles       1.00      0.89      0.94        18
      rec.sport.baseball       0.94      1.00      0.97        16
        rec.sport.hockey       1.00      1.00      1.00        18
               sci.crypt       0.92      1.00      0.96        22
         sci.electronics       0.93      0.64      0.76        22
                 sci.med       1.00      0.86      0.93        22
         

In [28]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

acc_score = accuracy_score(y_test,y_pred)
print('accuracy_score : ',acc_score)

prec_score = precision_score(y_test,y_pred , average = 'macro')
print('precision_score :',prec_score)

reca_score = recall_score(y_test,y_pred , average = 'macro')
print('recall_score : ',reca_score)

f_one_score = f1_score(y_test,y_pred ,average = 'macro')
print('f1_score : ', f_one_score)


accuracy_score :  0.84
precision_score : 0.8512275030785261
recall_score :  0.845803887309726
f1_score :  0.8393478607501523


conclusion:
1. the model trained good
2. the prediction with respect to testing data ,score is almost 90%

3. Sentiment analysis adds another layer of understanding, revealing the emotional tone of the blog posts. These insights can be crucial for content strategy, reader engagement, and marketing efforts.

4. By combining text classification with sentiment analysis, you gain a powerful toolkit for extracting and leveraging insights from textual data, enhancing both the analytical and strategic capabilities of your organization.

